In [ ]:
!pip install pyspark

In [1]:
from pyspark.sql import functions as F
from pyspark.sql import SparkSession
from pyspark.sql.types import FloatType, StringType

spark=SparkSession.builder.appName('FordGoBike').getOrCreate()
spark
# https://www.kaggle.com/code/priyankabnl/fordgobike-trip-data

In [7]:
path="G:/My Drive/Develhope/develhope-Data5-Team3/Data/2017-fordgobike-tripdata_final.csv"
df_pyspark=spark.read.csv(path,header=True, inferSchema=True)

df_pyspark=df_pyspark.drop('_c0')
df_pyspark.show()

+------------+--------------------+--------------------+----------------+--------------------+----------------------+-----------------------+--------------+--------------------+--------------------+---------------------+-------+----------+-----------------+-------------+-----------+
|duration_sec|          start_time|            end_time|start_station_id|  start_station_name|start_station_latitude|start_station_longitude|end_station_id|    end_station_name|end_station_latitude|end_station_longitude|bike_id| user_type|member_birth_year|member_gender|     pyment|
+------------+--------------------+--------------------+----------------+--------------------+----------------------+-----------------------+--------------+--------------------+--------------------+---------------------+-------+----------+-----------------+-------------+-----------+
|       80110|2017-12-31 16:57:...|2018-01-01 15:12:...|              74|Laguna St at Haye...|    37.776434819204745|    -122.42624402046204|       

In [4]:
# df_pyspark= df_pyspark.withColumn('start time hour',F.when(df_pyspark['start_am_pm']=='PM',df_pyspark["start time hour"]+12).otherwise(df_pyspark["start time hour"]))
# df_pyspark= df_pyspark.withColumn('start time hour',F.when(df_pyspark['start time hour']==24,12).otherwise(df_pyspark["start time hour"]))

In [5]:
# df_pyspark= df_pyspark.withColumn('end_time hour',F.when(df_pyspark['end_am_pm']=='PM',df_pyspark["end_time hour"]+12).otherwise(df_pyspark["end_time hour"]))
# df_pyspark= df_pyspark.withColumn('end_time hour',F.when(df_pyspark['end_time hour']==24,12).otherwise(df_pyspark["end_time hour"]))

In [ ]:
# df_pyspark.filter(F.col('end_time hour')==24).show()

In [6]:
#fixing 1-digit values 
# df_pyspark=df_pyspark.withColumn('start time hour',F.lpad(F.col('start time hour'),2,'0'))\
#     .withColumn('start time minute',F.lpad(F.col('start time minute'),2,'0'))\
#     .withColumn('start time seconds',F.lpad(F.col('start time seconds'),2,'0'))\
#     .withColumn('end_time hour',F.lpad(F.col('end_time hour'),2,'0'))\
#     .withColumn('end_time minute',F.lpad(F.col('end_time minute'),2,'0'))\
#     .withColumn('end_time seconds',F.lpad(F.col('end_time seconds'),2,'0'))

In [7]:
# #creating start time and end time columns with timestamp for easy comparison
# df_pyspark=df_pyspark.withColumn('start time',F.to_timestamp(F.concat_ws(':',F.lpad(F.col('start time hour'),2,'0'),F.lpad(F.col('start time minute'),2,'0'),F.lpad(F.col('start time seconds'),2,'0'))))
# df_pyspark=df_pyspark.withColumn('end time',F.to_timestamp(F.concat_ws(':',F.lpad(F.col('end_time hour'),2,'0'),F.lpad(F.col('end_time minute'),2,'0'),F.lpad(F.col('end_time seconds'),2,'0'))))

In [ ]:
df_pyspark.show()

In [8]:
#swapping end time and start time values if start time > end time
df_pyspark=df_pyspark.withColumn('end time',F.when(df_pyspark['end time']<df_pyspark['start time'], df_pyspark['start time'])\
                                 .otherwise(df_pyspark['end time']))
df_pyspark=df_pyspark.withColumn('start time',F.when(df_pyspark['end time']==df_pyspark['start time'], 
                                                     F.to_timestamp(F.concat_ws(':','end_time hour','end_time minute','end_time seconds')))\
                                 .otherwise(df_pyspark['start time']))

In [ ]:
# df_pyspark=df_pyspark.withColumn('start_time',F.concat_ws(':',F.lpad(F.hour(F.col('start time')),2,'0'),F.lapd(F.minute(F.col('start time')),2,'0'),F.lapd(F.second(F.col('start time')),2,'0')))#\
    #.withColumn('end_time',F.concat_ws(':',F.lpad(F.hour('end time'),2,'0'),F.lpad(F.minute('end time'),2,'0'),F.lpad(F.second('end time'),2,'0')))

In [9]:
df_pyspark=df_pyspark.withColumn('start_time',df_pyspark['start time']).withColumn('end_time',df_pyspark['end time'])

In [ ]:
df_pyspark.columns

In [ ]:
# #Dropping 'start time' and 'end time' columns
# df_pyspark=df_pyspark.drop(*['start time','end time','start time hour',
#  'start time minute',
#  'start time seconds',
#  'end_time hour',
#  'end_time minute',
#  'end_time seconds',])
# df_pyspark.show()

In [11]:
!pip install haversine


[notice] A new release of pip is available: 23.1.1 -> 23.1.2
[notice] To update, run: python -m pip install --upgrade pip


In [12]:
from haversine import haversine

def haversine_f(lat1, lon1, lat2, lon2):
    return haversine( (lat1, lon1), (lat2, lon2),unit='m',normalize=True )

haversine_udf = F.udf(haversine_f)

In [13]:
#Calculate haversine distance(Onur)
df_pyspark=df_pyspark.withColumn('haversine_distance', 
                    haversine_udf(F.col('start_station_latitude'), F.col('start_station_longitude'), 
                                  F.col('end_station_latitude'), F.col('end_station_longitude'))
                    )

In [12]:
#Calculate haversine distance in meters(Uros)
df_pyspark=df_pyspark.withColumn('haversine_distance',
                                 haversine_udf('start_station_latitude', 
                                               'start_station_longitude', 
                                               'end_station_latitude', 
                                               'end_station_longitude'))

In [14]:
#assign timestamp to start_time and end_time
#Calculate 'Diff_in_seconds' 
#Calculate 'Diff_in_minutes' 
#Calculate 'Trip_cost' 
df_pyspark=df_pyspark.withColumn('start_time',F.to_timestamp('start_time','HH:mm:ss'))\
    .withColumn('end_time',F.to_timestamp('end_time','HH:mm:ss'))\
    .withColumn('Diff_in_seconds',F.col('end_time').cast('long')-F.col('start_time').cast('long'))\
    .withColumn('Diff_in_minutes',(F.col('Diff_in_seconds')/60))\
    .withColumn('Trip_cost',(F.col('Diff_in_minutes')*0.35))

df_pyspark.printSchema()


root
 |-- start_time: timestamp (nullable = true)
 |-- start_am_pm: string (nullable = true)
 |-- end_time: timestamp (nullable = true)
 |-- end_am_pm: string (nullable = true)
 |-- start_station_id: integer (nullable = true)
 |-- start_station_name: string (nullable = true)
 |-- start_station_latitude: float (nullable = true)
 |-- start_station_longitude: float (nullable = true)
 |-- end_station_id: integer (nullable = true)
 |-- end_station_name: string (nullable = true)
 |-- end_station_latitude: float (nullable = true)
 |-- end_station_longitude: float (nullable = true)
 |-- bike_id: integer (nullable = true)
 |-- user_type: string (nullable = true)
 |-- member_birth_year: integer (nullable = true)
 |-- member_gender: string (nullable = true)
 |-- pyment: string (nullable = true)
 |-- haversine_distance: string (nullable = true)
 |-- Diff_in_seconds: long (nullable = true)
 |-- Diff_in_minutes: double (nullable = true)
 |-- Trip_cost: double (nullable = true)



In [15]:
df_pyspark.groupBy("bike_id")\
    .agg(F.sum("haversine_distance").alias("sum_distance")).sort(F.desc("sum_distance"))

DataFrame[bike_id: int, sum_distance: double]

In [ ]:
df_pyspark.show()

In [16]:
df_pyspark.write.option("header",True).mode('overwrite').csv('/workspaces/develhope-Data5-Team3/Data/FordGoBike')

23/05/09 08:46:17 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: start time hour, start time minute, start time seconds, , end_time hour, end_time minute, end_time seconds, , start_station_id, start_station_name, start_station_latitude, start_station_longitude, end_station_id, end_station_name, end_station_latitude, end_station_longitude, bike_id, user_type, member_birth_year, member_gender, pyment
 Schema: start time hour, start time minute, start time seconds, _c4, end_time hour, end_time minute, end_time seconds, _c9, start_station_id, start_station_name, start_station_latitude, start_station_longitude, end_station_id, end_station_name, end_station_latitude, end_station_longitude, bike_id, user_type, member_birth_year, member_gender, pyment
Expected: _c4 but found: 
CSV file: file:///workspaces/develhope-Data5-Team3/Data/2017-fordgobike-tripdataa.csv


In [ ]:
df_pyspark.show()